##### Secret Key and Scope

In [0]:
df=dbutils.secrets.get(scope="key-vault-scope",key="app-secrect-key")

In [0]:
dbutils.secrets.listScopes()

##### Azure Datalake storage with Databricks

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": "56c1b601-2d0b-456e-966f-5153657934c6",
          "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="key-vault-scope",key="app-secrect-key"),
          "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/fdc798aa-6f15-4e72-bf4a-6c813690fc75/oauth2/token"}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = "abfss://tokyo-olympics@adlsgen2tokyoolympics.dfs.core.windows.net/",
  mount_point = "/mnt/tokyo-olympics-data",
  extra_configs = configs)

In [0]:
%fs
ls mnt/tokyo-olympics-data

In [0]:
%fs
ls mnt/tokyo-olympics-data/raw-data

###Tokyo Olmpics Transformation

#####"Medals Overview" Transformations

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum, rank
from pyspark.sql.window import Window

# Create Spark session (if not already created)
spark = SparkSession.builder.appName("TokyoOlympics").getOrCreate()

# Load the medals table from the mounted Parquet path
medals_path = "/mnt/tokyo-olympics-data/raw-data/medals.parquet"
medals_df = spark.read.format("parquet").load(medals_path)
medals_df.show()
# Transformation 1: Aggregate medal counts by TeamNOC
medals_summary = medals_df.groupBy("TeamNOC").agg(
    _sum("Gold").alias("TotalGold"),
    _sum("Silver").alias("TotalSilver"),
    _sum("Bronze").alias("TotalBronze"),
    _sum("Total").alias("TotalMedals")
)

# Transformation 2: Rank countries by Total Medals and Gold Medals
windowSpecTotal = Window.orderBy(col("TotalMedals").desc())
windowSpecGold = Window.orderBy(col("TotalGold").desc())

ranked_medals = medals_summary.withColumn("RankByTotalMedals", rank().over(windowSpecTotal)) \
                               .withColumn("RankByGoldMedals", rank().over(windowSpecGold))

# Save the transformed data back to a Parquet file in ADLS
output_path = "/mnt/tokyo-olympics-data/transformed-data/medals_summary"
ranked_medals.write.format("parquet").mode("overwrite").save(output_path)

# Display the final DataFrame
ranked_medals.show()


In [0]:
%fs
ls /mnt/tokyo-olympics-data/transformed-data

#####"Athletes Participation" Transformations

In [0]:
from pyspark.sql.functions import countDistinct

# Load the athletes table from the mounted Parquet path
athletes_path = "/mnt/tokyo-olympics-data/raw-data/athletes.parquet"
athletes_df = spark.read.format("parquet").load(athletes_path)

# Transformation 1: Count athletes grouped by NOC and Discipline
athletes_summary = athletes_df.groupBy("NOC", "Discipline").agg(
    countDistinct("Name").alias("TotalAthletes")
)

# Transformation 2: Get distinct athletes with their NOC and Discipline
distinct_athletes = athletes_df.select("Name", "NOC", "Discipline").distinct()

# Save the transformed data back to ADLS
athletes_summary_path = "/mnt/tokyo-olympics-data/transformed-data/athletes_summary"
distinct_athletes_path = "/mnt/tokyo-olympics-data/transformed-data/distinct_athletes"

athletes_summary.write.format("parquet").mode("overwrite").save(athletes_summary_path)
distinct_athletes.write.format("parquet").mode("overwrite").save(distinct_athletes_path)

# Display the results
athletes_summary.show()
distinct_athletes.show()


#####"Gender Distribution" Transformations

In [0]:
from pyspark.sql.functions import col, expr

# Load the entriesgender table from the mounted Parquet path
entriesgender_path = "/mnt/tokyo-olympics-data/raw-data/entriesgender.parquet"
entriesgender_df = spark.read.format("parquet").load(entriesgender_path)

# Transformation 1: Aggregate gender-wise participation by Discipline
gender_summary = entriesgender_df.select( \
    "Discipline", \
    col("Female").cast("int").alias("FemaleParticipants"), \
    col("Male").cast("int").alias("MaleParticipants"), \
    col("Total").cast("int").alias("TotalParticipants") \
)

# Transformation 2: Calculate gender ratio (Female and Male percentage)
gender_ratio = gender_summary.withColumn( \
    "FemalePercentage", expr("ROUND((FemaleParticipants / TotalParticipants) * 100, 2)") \
).withColumn( \
    "MalePercentage", expr("ROUND((MaleParticipants / TotalParticipants) * 100, 2)") \
)

# Save the transformed data back to ADLS
gender_summary_path = "/mnt/tokyo-olympics-data/transformed-data/gender_summary"

gender_ratio.write.format("parquet").mode("overwrite").save(gender_summary_path)

# Display the results
gender_ratio.show()
